In [7]:
html_text = ""

with open(
    file=r"notebooks\pasta teste\Devoluções.htm", mode="r", encoding="utf-8"
) as f:
    html_text = f.read()

In [104]:
all_tables = [
    (t[t.find("<table") :] + "</table>")
    .replace("<tr >", "<tr>")
    .replace("<th >", "<th>")
    .replace("<td >", "<td>")
    .replace("th>", "td>")
    for t in html.split("</table>")
]

In [5]:
# Função que isola a(s) tabela(s)
def get_tables(html: str):
    tables = [
        (t[t.find("<table") :] + "</table>")
        .replace("<tr >", "<tr>")
        .replace("<th >", "<th>")
        .replace("<td >", "<td>")
        .replace("th>", "td>")
        for t in html.split("</table>")
    ]
    if tables:
        return tables[:-1]
    return None

In [ ]:
def convert_to_md_table(df: pd.DataFrame) -> str:
    colunas = df.columns.tolist()

    texto = f"|{'|'.join(colunas)}|\n"
    texto += f"|{'|'.join(['---' for _ in range(len(colunas))])}|\n"

    for col in colunas:
        df[col] = df[col].fillna("")

    for _, l in df.iterrows():

        texto += f"|{'|'.join([str(l[c]) for c in colunas])}|\n"

    return texto

In [70]:
class DataFrame:
    def __init__(self, columns: list = [], rows: list = []):
        self.columns = columns
        self.rows = rows

    columns = []
    rows = []

    def to_md(self) -> str:

        md_text = ""
        if self.columns in [None, []]:
            return md_text

        # Titulos
        md_text += f"|{'|'.join(self.columns)}|\n"
        md_text += f"|{'|'.join(['---' for _ in self.columns])}|\n"

        if self.rows not in [None, []]:
            md_text += "\n".join([f"|{'|'.join(r)}|" for r in self.rows])

        return md_text

    def to_html(self) -> str:

        md_text = "<table>\n"
        if self.columns in [None, []]:
            return md_text

        # Titulos
        md_text += (
            f"<thead>\n<tr><th>{'</th><th>'.join(self.columns)}</th></tr>\n</thead>\n"
        )

        if self.rows not in [None, []]:
            md_text += "<tbody>\n"
            md_text += "\n".join(
                [f"<tr><td>{'</td><td>'.join(r)}</td></tr>" for r in self.rows]
            )
            md_text += "\n</tbody>\n"

        md_text += "</table>"

        return md_text

    @staticmethod
    def from_html(html_table: str):
        tabelas = DataFrame.__get_tables(html_table)
        if tabelas is None:
            return DataFrame()
        else:
            if len(tabelas) == 1:
                tabela = tabelas[0]
                columns, rows = DataFrame.__get_one_table(tabela)
                return DataFrame(columns=columns, rows=rows)
            else:

                dfs = []

                for t in tabelas:
                    columns, rows = DataFrame.__get_one_table(t)
                    df = DataFrame(columns, rows)
                    dfs.append(df)

                return dfs

    @staticmethod
    def __get_tables(html: str):
        tables = [
            (t[t.find("<table") :] + "</table>")
            .replace("<tr >", "<tr>")
            .replace("<th >", "<th>")
            .replace("<td >", "<td>")
            .replace("th>", "td>")
            for t in html.split("</table>")
        ]
        if tables:
            return tables[:-1]
        return None

    @staticmethod
    def __get_one_table(html_table):
        linhas = [l[l.find("<tr>") + 4 :] for l in html_table.split("</tr>")][:-1]
        columns = None
        rows = None

        if len(linhas) > 0:
            columns = [
                t.replace("<td>", "").replace("</td>", "")
                for t in linhas[0].split("</td><td>")
            ]

        if len(linhas) > 1:
            rows = [
                [
                    l.replace("<td>", "").replace("</td>", "")
                    for l in t.split("</td><td>")
                ]
                for t in linhas[1:]
            ]

        return columns, rows

In [71]:
print(DataFrame.from_html(html_text)[0].to_html())

<table>
<thead>
<tr><th>DisplayName</th><th>Name</th><th>Ordinal</th><th>DisplayOption</th><th>Width</th><th>Height</th><th>PageIndex</th></tr>
</thead>
<tbody>
<tr><td>Home</td><td>ReportSectionf006ae245620d788e624</td><td>0</td><td>1</td><td>1920</td><td>1080</td><td>0</td></tr>
<tr><td>Relatório</td><td>ReportSectiona2845233991e3e9977b4</td><td>1</td><td>1</td><td>1920</td><td>1080</td><td>1</td></tr>
</tbody>
</table>


In [77]:
linhas = [l[l.find("<tr>") + 4 :] for l in table.split("</tr>")][:-1]
titulos = None
dados = None

if len(linhas) > 0:
    titulos = [
        t.replace("<td>", "").replace("</td>", "") for t in linhas[0].split("</td><td>")
    ]

if len(linhas) > 1:
    dados = [
        [l.replace("<td>", "").replace("</td>", "") for l in t.split("</td><td>")]
        for t in linhas[1:]
    ]

In [72]:
dados[-1]

['shape',
 '',
 '0',
 '136,105860113422',
 '2000',
 '1920',
 '96,1814744801512',
 '27f55178e74399b0995c',
 '14000',
 '14',
 '1']

In [59]:
import pandas as pd

In [60]:
pd.DataFrame(dados, columns=titulos)

,Visual Type,Title,X,Y,Z,Width,Height,Name,tabOrder,VisualIndex,PageIndex
0,image,,0,0,3000,"178,83149872989","149,56816257409",f3c7cb5da0b360834180,,0,1
1,shape,,"1717,80612244898",0,0,"170,816326530612","56,4795918367347",5fc1d20717514e7dc1ac,6000,1,1
2,textbox,,"1717,80612244898","5,51020408163265",4000,"170,816326530612","50,969387755102",783149ec03d257193a8c,1000,2,1
3,actionButton,,"1717,80612244898",0,6000,"170,816326530612","56,4795918367347",57778428d48d27bce478,3000,3,1
4,ChicletSlicer1448559807354,'Canal',0,"212,325141776938",7000,"282,878916172735","89,415749364945",3eec4430b8be50aee3d0,4000,4,1
5,ChicletSlicer1448559807354,'UF',283,"212,325141776938",9000,239,88,9c4872d095ed4450a7b0,7000,5,1
6,ChicletSlicer1448559807354,'Tipo de devolução',522,"212,325141776938",10000,343,88,734deacd23a0a7557548,9000,6,1
7,tableEx,,"9,07372400756144","342,986767485822",11000,"1900,03780718336","736,786389413989",2511e9b012d05655a3a7,10000,7,1
8,ChicletSlicer1448559807354,'Ano',1160,"212,325141776938",12000,243,88,b8a5598a320ab27410d0,11000,8,1
9,ChicletSlicer1448559807354,'Mês',"1402,797731569","212,325141776938",13000,"517,202268431002","87,1077504725898",9c7a895341a2abdbb34e,12000,9,1
